# follow along with meme generator here:
https://towardsdatascience.com/meme-text-generation-with-a-deep-convolutional-network-in-keras-tensorflow-a57c6f218e85

In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
import math 
import re
import operator
import time

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import utils

In [5]:
cwd = os.getcwd()
print(cwd)
parent_wd = cwd.replace('/names', '')
raw_data_path = os.path.join(parent_wd, 'raw_data/moonGen_scrape_2016')

/Users/markliu/moonboard/names


In [6]:
raw_data_path

'/Users/markliu/moonboard/raw_data/moonGen_scrape_2016'

In [7]:
with open(raw_data_path, 'rb') as f:
    raw_data = pickle.load(f)

In [8]:
print(list(raw_data.items())[3][1])

{'problem_name': 'GULAG', 'info': ['micklevin', '1 climber has repeated this problem', '7A', 'Feet follow hands', ''], 'url': 'https://moonboard.com/Problems/View/367885/gulag', 'num_empty': 1, 'num_stars': 2, 'moves': [{'Id': 2119166, 'Description': 'A5', 'IsStart': True, 'IsEnd': False}, {'Id': 2119167, 'Description': 'B8', 'IsStart': False, 'IsEnd': False}, {'Id': 2119168, 'Description': 'D12', 'IsStart': False, 'IsEnd': False}, {'Id': 2119169, 'Description': 'F14', 'IsStart': False, 'IsEnd': False}, {'Id': 2119170, 'Description': 'I18', 'IsStart': False, 'IsEnd': True}], 'grade': '7A', 'UserGrade': None, 'isBenchmark': False, 'repeats': 1, 'ProblemType': None, 'IsMaster': False, 'setter': {'Id': '05f91964-f58c-4a49-af50-abec02f71300', 'Nickname': 'micklevin', 'Firstname': 'Mick', 'Lastname': 'Levin', 'City': 'Halifax', 'Country': 'Canada', 'ProfileImageUrl': '/Content/Account/Users/Profile/05f91964-f58c-4a49-af50-abec02f71300.jpg?637231996827206508', 'CanShareData': True}}


In [10]:
problem = list(raw_data.items())[3][1]
utils.convert_problem_to_name_input(problem['moves'])

'A5,B8,D12,F14,I18'

In [11]:
# look at all the problem name characters
names = []
i = 0
for id_problem in list(raw_data.items()):
#     if i !=0:
#         break
    i+=1
    problem = id_problem[1]
    name = problem['problem_name']
    #print(name)
    ignore = False
    for ch in name:
        if ch >= 'A' and ch <= 'Z' or ch.isspace() or ch >= '0' and ch <= '9':
            #print(chars)
            continue
        else:
            ignore = True
            break
    if not ignore:
        names.append(name)
    

In [12]:
len(names)

24793

In [20]:
# do all the names have a common starting letter?
counts = {}
for name in names:
    if name[0] != 'S' or len(name) <= 1:
        continue
    k = name[1]
    counts[k] = counts[k] + 1 if k in counts else 1
print(counts)

{'I': 186, 'A': 249, 'M': 106, 'L': 90, 'T': 630, 'C': 120, 'U': 211, 'Y': 24, 'P': 177, 'E': 177, 'H': 281, 'K': 56, 'O': 223, 'Q': 48, 'J': 2, 'W': 94, 'N': 78, 'V': 9, 'G': 5, 'B': 10, 'F': 9, 'S': 5, 'D': 1, 'R': 8, 'X': 4, '3': 2, ' ': 2, 'Z': 1, '6': 1, '5': 1, '4': 1, '2': 1, '1': 1, '7': 1}


In [10]:
len(list(raw_data.items()))

30673

In [11]:
def validChar(ch):
    return ch >= 'A' and ch <= 'Z' or ch.isspace() or ch >= '0' and ch <= '9'

def validProblemName(name):
    return all([validChar(ch) for ch in name])

def buildInputForSingleProblem(problem, inp):
    """Continue to append to inp with all the inputs from problem"""
    holds_str = convert_problem_to_input(problem['moves'])
    name = problem['problem_name']
    # artificially add end of string char
    name+='|'
    for i in range(0, len(name)):
        x = (holds_str+" "+name[0:i]).upper()
        y = name[i].upper()
        inp.append([x,y])

def buildInputsFromProblems(raw_data):
    inp = []
    for id_problem in list(raw_data.items()):
        try:
            problem = id_problem[1]
            name = problem['problem_name']
            if not validProblemName(name):
                continue
            buildInputForSingleProblem(problem, inp)
        except Exception as e:
            print("failed to convert problem ", id_problem[0])
            print(e)
    return inp
        
    

In [12]:
# inp = buildInputsFromSingleProblem(problem, inp)
inp = buildInputsFromProblems(raw_data)

failed to convert problem  348825
'moves'


In [13]:
print(len(inp))
print(len(list(raw_data.items())))
#print(problem)

308455
30673


In [14]:
texts = [row[0] for row in inp]
labels = [row[1] for row in inp]

In [15]:
labels

['T',
 'A',
 'L',
 'L',
 ' ',
 'P',
 'O',
 'P',
 'P',
 'Y',
 '|',
 'C',
 'A',
 'T',
 'S',
 ' ',
 'A',
 'R',
 'E',
 ' ',
 'F',
 'U',
 'R',
 'R',
 'Y',
 ' ',
 'C',
 'H',
 'I',
 'L',
 'D',
 'R',
 'E',
 'N',
 '|',
 'L',
 'I',
 'F',
 'E',
 ' ',
 'I',
 'S',
 ' ',
 'S',
 'U',
 'F',
 'F',
 'E',
 'R',
 'I',
 'N',
 'G',
 '|',
 'G',
 'U',
 'L',
 'A',
 'G',
 '|',
 'J',
 'U',
 'S',
 'T',
 ' ',
 'K',
 'E',
 'E',
 'P',
 ' ',
 'S',
 'W',
 'I',
 'M',
 'M',
 'I',
 'N',
 'G',
 '|',
 'J',
 'U',
 'S',
 'T',
 ' ',
 'K',
 'E',
 'E',
 'P',
 ' ',
 'S',
 'W',
 'I',
 'M',
 'M',
 'I',
 'N',
 'G',
 '|',
 'M',
 'E',
 'T',
 'E',
 'M',
 'P',
 'S',
 'I',
 'C',
 'O',
 'S',
 'I',
 '|',
 'L',
 'A',
 ' ',
 'C',
 'R',
 'I',
 'S',
 'I',
 'S',
 ' ',
 'D',
 'E',
 'L',
 ' ',
 'P',
 'A',
 'P',
 'E',
 'L',
 '|',
 'R',
 'A',
 'R',
 'E',
 '|',
 'I',
 'S',
 'O',
 'L',
 'A',
 'T',
 'I',
 'O',
 'N',
 ' ',
 'C',
 'C',
 'C',
 'C',
 'C',
 '|',
 'H',
 'E',
 'A',
 'V',
 'Y',
 ' ',
 'M',
 'O',
 'N',
 'D',
 'A',
 'Y',
 '|',
 'E',
 'S',
 'C'

In [16]:
# TODO - data cleaning

In [17]:
# restructure as tensors
from keras import Sequential
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.layers import BatchNormalization
import numpy as np


In [19]:
char_to_int, int_to_char = utils.map_char_to_int(texts, labels)
sequences = utils.texts_to_sequences(texts, char_to_int)
labels = utils.[char_to_int[char] for char in labels]

In [20]:
char_to_int

{',': 1,
 '1': 2,
 'E': 3,
 ' ': 4,
 '8': 5,
 'G': 6,
 '5': 7,
 'I': 8,
 'F': 9,
 'D': 10,
 'H': 11,
 'A': 12,
 'C': 13,
 '4': 14,
 '3': 15,
 '6': 16,
 'B': 17,
 '0': 18,
 '2': 19,
 'O': 20,
 'K': 21,
 'T': 22,
 'J': 23,
 'R': 24,
 '9': 25,
 'S': 26,
 'N': 27,
 '7': 28,
 'L': 29,
 'M': 30,
 'U': 31,
 'P': 32,
 'Y': 33,
 'W': 34,
 '|': 35,
 'V': 36,
 'Z': 37,
 'X': 38,
 'Q': 39}

In [21]:
int_to_char

['?',
 ',',
 '1',
 'E',
 ' ',
 '8',
 'G',
 '5',
 'I',
 'F',
 'D',
 'H',
 'A',
 'C',
 '4',
 '3',
 '6',
 'B',
 '0',
 '2',
 'O',
 'K',
 'T',
 'J',
 'R',
 '9',
 'S',
 'N',
 '7',
 'L',
 'M',
 'U',
 'P',
 'Y',
 'W',
 '|',
 'V',
 'Z',
 'X',
 'Q']

In [22]:
#longest_seq = max(sequences, key = lambda i: len(i))
#len(longest_seq) = 102

In [25]:
SEQUENCE_LENGTH = 128
data = pad_sequences(sequences, maxlen=SEQUENCE_LENGTH)
labels = np.array(labels)

In [28]:
# randomize order of training data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [ ]:
# double check
[int_to_char[i] for i in data[0]]
[int_to_char[labels[0]]]

In [ ]:
data.shape

In [29]:
validation_ratio = 0.2 if data.shape[0] < 1000000 else 0.02
num_validation_samples = int(validation_ratio * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

# Model
copying model from https://towardsdatascience.com/meme-text-generation-with-a-deep-convolutional-network-in-keras-tensorflow-a57c6f218e85

In [30]:
EMBEDDING_DIM = 16
model = Sequential()
model.add(Embedding(len(char_to_int) + 1, EMBEDDING_DIM, input_length=SEQUENCE_LENGTH))
model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(len(char_to_int)+1, activation='softmax')) # kinda hacky?

model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [33]:
import json
MODEL_PATH = 'moonboard_names_model_128'

with open(MODEL_PATH + '/params.json', 'w') as handle:
    json.dump({
        'sequence_length': 128,
        'embedding_dim': EMBEDDING_DIM,
        'num_rows_used': len(sequences),
        'num_epochs': 48,
        'batch_size': 256,
        'char_to_int': char_to_int,
        'int_to_char': int_to_char
    }, handle)

In [ ]:
# the path where you want to save all of this model's files
MODEL_PATH = 'moonboard_names_model'
# just make this large since you can stop training at any time
NUM_EPOCHS = 48
# batch size below 256 will reduce training speed since
# CPU (non-GPU) work must be done between each batch
BATCH_SIZE = 256
# callback to save the model whenever validation loss improves
checkpointer = ModelCheckpoint(filepath=MODEL_PATH + '/model.h5', verbose=1, save_best_only=True)
# custom callback to save history and plots after each epoch
history_checkpointer = SaveHistoryCheckpoint(MODEL_PATH)
# the main training function where all the magic happens!
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[checkpointer, history_checkpointer])